In [2]:
import sys
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from konlpy.tag import Mecab
import pandas as pd
import pickle

In [35]:
class Text_Summarizer():
    def __init__(self):
        
        self.mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")
        #rank높은 문장 갯수
        self.top_n = 3
        
        ##불용어 사전
        stop_words_f ="./stopwords.txt"
        with open(stop_words_f, "r", encoding='utf-8') as f:
             stop_words = f.readlines()
        self.stop_words = [stop_word.strip() for stop_word in stop_words]
        
        
    #문장 cleaning
    def cleaning(self, sentences):
        for i in range(len(sentences)):
            sentences[i] = re.sub('\《.*\》|\s-\s.*', '', sentences[i])
            sentences[i] = re.sub('\(.*\)|\s-\s.*', '', sentences[i])
            #필드의 태그를 모두 제거
            sentences[i] = re.sub('(<([^>]+)>)', '', sentences[i])
            # 개행문자 제거
            sentences[i] = re.sub('\n', ' ', sentences[i])
            #특수문자 제거
            sentences[i] = re.sub(r'[^\w\s]', '', sentences[i])
        return sentences    
    
    #mecab으로 조사, 전치사 분리
    def mecab_morphs(self, sentences):
        for i, sentence in enumerate(sentences):
            sentences[i] = self.mecab.morphs(sentence)
        return sentences

    #mecab으로 #일반명사 #고유명사 #대명사 #동사 #형용사
    def mecab_pos(self, sentences):
        
        for i, sentence in enumerate(sentences):
            sent_pos = []
            for word in self.mecab.pos(sentence):
                if word[1] in ['NNG', 'NNP','NP','VV','VA' ]:
                    sent_pos.append(word[0])
            sentences[i] = sent_pos
        return sentences
    
    def sentence_similarity(self, sent1, sent2):
        
        #토큰이 비어있으면 제거
        for i in range(len(sent1)):
            if range(len(sent1[i])) == 0:
                sent1.pop(i)
        for i in range(len(sent2)):
            if range(len(sent2[i])) == 0:
                sent2.pop(i)
        #비교할 문장들의 토큰합
        all_words = list(set(sent1 + sent2))
        
        #cosine 유사도를 구할 vector 생성
        vector1 = [0] * len(all_words)
        vector2 = [0] * len(all_words)
        
        # 첫 문장 생성
        for w in sent1:
            if w in self.stop_words:
                continue
            vector1[all_words.index(w)] += 1
        # 두번째 문장 생성
        for w in sent2:
            if w in self.stop_words:
                continue
            vector2[all_words.index(w)] += 1
        
        #유사도 계산
        return 1 - cosine_distance(vector1, vector2)
    

    def graph_draw(self):
        #각 노드는 각 문장을 의미 엣지의 두께는 연결된 노드들의 코사인 유사도
        size = 10
        size_similarity_matrix = self.similarity_matrix.flatten()*size
        nx.draw(self.G, with_labels=True, node_color='white', width=size_similarity_matrix.tolist())


    def __call__(self, document, morphs=False):
        
        #self.document = document.split(". ")
        #최종 output으로 나올 문서 백업
        sentences = document.copy()
        
        c_sentences = self.cleaning(document)

        if morphs == True:
            morph_c_sentences = self.mecab_morphs(c_sentences)
        else:
            morph_c_sentences = self.mecab_pos(c_sentences)
        
        #유사도행렬 구성
        self.similarity_matrix = np.zeros((len(sentences), len(sentences)))
        for idx1 in range(len(sentences)):
            for idx2 in range(len(sentences)):
                if idx1 == idx2: #같은 문장은 계산하지않습니다.
                    continue 
                self.similarity_matrix[idx1][idx2] = self.sentence_similarity(morph_c_sentences[idx1], morph_c_sentences[idx2])
    
        #유사도행렬을 인접행렬로 취급 무방향 그래프를 그린다.
        self.G = nx.from_numpy_array(self.similarity_matrix)
        #유사도 행렬로 pagerank 알고리즘 적용
        scores = nx.pagerank(self.G)
        #랭크 높은순으로 최종 요약 제공
        ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
        summarize_text = []
        for i in range(self.top_n):
          summarize_text.append("".join(ranked_sentence[i][1]))
    
        return " ".join(summarize_text)

In [36]:
#inference
file_name = "test1.txt"
file = open(file_name, "r", encoding="UTF-8")
document = file.read()
article = document.split(". ")
article

['[보안뉴스 문정후 기자] 인공지능이라는 신기술이 기업들의 고객 대응 방법과 사업 운영 방향성을 크게 바꿔놓고 있다',
 '간단하고 반복적인 임무를 훨씬 정확하고 빠르게 처리해 주고, 수많은 데이터 안에 숨겨져 있어 놓칠 뻔한 통찰을 찾아내 주며, 그러므로 보다 시기적절하고 좋은 결과를 낳는 결정을 내릴 수 있게 해 준다',
 '그렇지만 인공지능을 주류로 올리기에는 아직까지 해소되지 않은 염려거리들이 있다\n.인간이 아닌 기계가 데이터를 분석하고 결정까지 내린다고 했을 때 가장 먼저 생각나는 문제점은 ‘윤리성’이다',
 '기계의 결정이 윤리적으로 그릇된 것일 때가 있다는 건 이미 실제 실험과 프로젝트를 통해 입증되기도 했었다',
 '그래서 인공지능의 성능과 장점에만 집중하던 기업들은 어느 순간부터 인공지능의 윤리성에도 똑같이 관심을 갖기 시작했다.\n\n“인공지능은 강력한 신기술로 장점들만 열거해고 긴 목록이 만들어집니다',
 '하지만 이 장점들을 빠짐없이 누리기 위해서는 신뢰의 시스템을 먼저 구축할 필요가 있습니다',
 '인공지능이라는 기술 자체의 신뢰도도 높이고, 인공지능을 개발하는 사람이나 업체의 신뢰도도 높아져야 합니다',
 '인공지능의 결정을 설명할 수 있어야 하고, 인공지능이 데이터를 어떻게 처리하는지도 투명하게 공개해야 하며, 편견이 반영되지 않도록 하는 작업이 필요합니다.” IBM의 인공지능 윤리 책임자인 프란체스카 로시(Francesca Rossi)의 설명이다.\n\n“인공지능의 윤리 문제는 ‘인공지능이 사회에 미칠 영향’이라는 관점에서 이해되어야 합니다',
 '의도치 않은 사회적 악영향들은 최소화 하면서 전 세계적인 기술 혁신은 이어지도록 해야 하는 것이죠.” 윤리 자문 기업인 에시컬인텔리전스(Ethical Intelligence)의 CEO 올리비아 갬벨린(Olivia Gambelin)의 설명이다',
 '“인공지능 윤리를 실제 운영 환경에 적용한다는 건 상위 개념의 원리들을 구체적이고 세부적인 행동 지침으로 전환하여 인간 중심의 핵심 가치를

In [37]:
text_summarize = Text_Summarizer()
summarize = text_summarize(article)
print(summarize)

인공지능의 결정을 설명할 수 있어야 하고, 인공지능이 데이터를 어떻게 처리하는지도 투명하게 공개해야 하며, 편견이 반영되지 않도록 하는 작업이 필요합니다.” IBM의 인공지능 윤리 책임자인 프란체스카 로시(Francesca Rossi)의 설명이다.

“인공지능의 윤리 문제는 ‘인공지능이 사회에 미칠 영향’이라는 관점에서 이해되어야 합니다 오히려 그런 규정과 표준들이 정립됨으로써 인공지능 기술 혁신이 가속화 될 거라고 개인적으로 믿고 있습니다.”

인공지능 윤리, 앞으로 어떻게 될까
그렇기 때문에 인공지능 기술들을 개발하는 조직들이라면 시작부터 인공지능 윤리 문제를 같이 고민해야 한다고 PwC의 글로벌 인공지능 담당인 아난드 라오(Anand Rao)는 강조한다 실험을 하는 것도 대단히 위험하고요.”

인공지능 윤리 문제 처리하기
이렇게 인공지능을 불안하게 만드는 많은 요인들 중 하나가 ‘인공지능 윤리’다


In [ ]:
#inference
file_name = "test1.txt"
file = open(file_name, "r", encoding="UTF-8")
document = file.read()
article = document.split(". ")


text_summarize.graph_draw()

In [25]:
#사설 문서 요약 검증
def validation_result(morphs = False):
    val_df = pd.read_pickle("val_df.pkl")

    references = []
    for i in range(len(val_df.extractive)):
        val = " ".join(val_df.extractive[i])
        references.append(val)

    #검증 데이터 모델 추론
    text_sumarize = Text_Summarizer()
    inferences = []
    for i in range(len(val_df.text)):
        result = text_sumarize(val_df.text[i], morphs=morphs)
        inferences.append(result)
        print("{a}경과중".format(a=i))

    #추론 리스트와 정답 리스트의 길이 확인
    print(f'inferences_length: {len(inferences)}, references_length: {len(references)}')

    return inferences, references

In [26]:
from rouge import Rouge
rouge = Rouge()

In [ ]:
inferences, references = validation_result(morphs = False)
scores = rouge.get_scores(inferences, references, avg=True)

In [27]:
with open('model_2_valdata_score.pkl','rb') as f:
    scores = pickle.load(f)

In [29]:
print(" Rouge-1: recall: {:.3f} precision: {:.3f} f-1: {:.3f}".format(scores['rouge-1']['r'], scores['rouge-1']['p'], scores['rouge-1']['f']),'\n',
      "Rouge-2: recall: {:.3f} precision: {:.3f} f-1: {:.3f}".format(scores['rouge-2']['r'], scores['rouge-2']['p'], scores['rouge-2']['f']),'\n',
      "Rouge-L: recall: {:.3f} precision: {:.3f} f-1: {:.3f}".format(scores['rouge-l']['r'], scores['rouge-l']['p'], scores['rouge-l']['f']))

 Rouge-1: recall: 0.337 precision: 0.335 f-1: 0.332 
 Rouge-2: recall: 0.288 precision: 0.280 f-1: 0.281 
 Rouge-L: recall: 0.336 precision: 0.333 f-1: 0.330
